In [6]:
#UI to ask the query
input_query=""

In [7]:
#Keyword Extraction of the search query
search_term=""

In [8]:
! pip install apify_client re

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [1]:
import pandas as pd
import json

# Read the JSON file into a pandas DataFrame
json_file_path = 'government_schemes.json'

with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

# Create lists to store data
policy_name_list = []
ministry_list = []
state_list = []
details_list = []
benefits_list = []
eligibility_list = []

# Dictionary to keep track of policy names and their counts
policy_name_counts = {}

df=pd.read_json("government_schemes.json")
print(df.info())
print(df.shape)
print(df.head(5))
# Iterate through each policy
for policy in data:
    policy_name = policy['Policy name']
    ministry = policy['Ministry']
    state = policy['State']
    details = policy['Description']['details']
    benefits = policy['Description']['benefits']
    eligibility = policy['Description']['eligibility']

    # Count occurrences of policy name
    if policy_name in policy_name_counts:
        policy_name_counts[policy_name] += 1
    else:
        policy_name_counts[policy_name] = 1

print(policy_name_counts)
already_listed_policies=[]
# Iterate through each policy again to determine "Central" policies
for policy in data:
    policy_name = policy['Policy name'].strip()
    ministry = policy['Ministry'].strip()
    state = policy['State'].strip()
    details = policy['Description']['details'].strip()
    benefits = policy['Description']['benefits'].strip()
    eligibility = policy['Description']['eligibility'].strip()

    # If policy name has more than two occurrences, add one to "Central" and remove extra occurrences
    if policy_name_counts[policy_name] > 1:
        if policy_name not in already_listed_policies :  # Only add to "Central" once
            policy_name_list.append(policy_name)
            ministry_list.append(ministry)
            state_list.append('Central')
            details_list.append(details)
            benefits_list.append(benefits)
            eligibility_list.append(eligibility)
            already_listed_policies.append(policy_name)
        else:
            pass

    else:
        # For policies with one occurrence, add normally
        policy_name_list.append(policy_name)
        ministry_list.append(ministry)
        state_list.append(state)
        details_list.append(details)
        benefits_list.append(benefits)
        eligibility_list.append(eligibility)

# Create a DataFrame
df = pd.DataFrame({
    'Policy name': policy_name_list,
    'Ministry': ministry_list,
    'State': state_list,
    'Description': details_list,
    'Benefits': benefits_list,
    'Eligibility': eligibility_list
})

missing_values = df.isnull().sum()
print("Missing values per column in main df:")
print(missing_values)

df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
print(df.info())
print(df.shape)
print(df.head())
# Specify the CSV file name
csv_file = 'cleand_government_schemes.csv'

# Write the DataFrame to a CSV file
df.to_csv(csv_file, index=False)
print("CSV conversion complete. Modified data saved to 'government_schemes.csv'.")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1810 entries, 0 to 1809
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Sno          1810 non-null   int64 
 1   Policy name  1810 non-null   object
 2   Description  1810 non-null   object
 3   Ministry     1810 non-null   object
 4   State        1810 non-null   object
dtypes: int64(1), object(4)
memory usage: 70.8+ KB
None
(1810, 5)
   Sno                                        Policy name  \
0    1  National AIDS Control Organisation (NACO) Inte...   
1    2                Quality Improvement Programme (QIP)   
2    3              Livestock Health and Diseases Control   
3    4  Financial Assistance For Widows, Divorced, Sep...   
4    5                 Ramalingaswami Re-Entry Fellowship   

                                         Description  \
0  {'details': 'NACO initiated the Internship Pro...   
1  {'details': 'The scheme "Quality Improvement P...   
2  {'d

In [10]:
# Function to get unique policy names and select minimum 10
def get_unique_policy_names(group):
    unique_policy_names = set()  # Using a set to store unique policy names
    policy_names = []
    for policy in group['Policy name']:
        if policy not in unique_policy_names:
            unique_policy_names.add(policy)
            policy_names.append(policy)
        if len(policy_names) >= 20:
            break

    return pd.DataFrame({
        'State': [group['State'].iloc[0]] * len(policy_names),
        'Policy name': policy_names
    })

# Group by 'State', get unique policy names, and select minimum 10 for each state
policy_names_min_20 = df.groupby('State').apply(get_unique_policy_names).reset_index(drop=True)

# Save the result DataFrame to a new CSV file
csv_file_result = 'min_20_unique_policy_names_according_to _state.csv'
policy_names_min_20.to_csv(csv_file_result, index=False)

print("CSV conversion complete. DataFrame with minimum 10 unique policy names for each state (without duplicates), including state, saved to 'government_schemes_min_20_per_state_unique.csv'.")

CSV conversion complete. DataFrame with minimum 10 unique policy names for each state (without duplicates), including state, saved to 'government_schemes_min_20_per_state_unique.csv'.


In [11]:
import re

def generate_policy_name_variations(policy_name):
    variations = []
    variations.append(policy_name)
    # Function to generate variations by removing one or two last words
    def generate_variations(words):
        variations = []
        if len(words) >= 2:
            variations.append(' '.join(words[:-1]))  # Remove last word
        if len(words) >= 3:
            variations.append(' '.join(words[:-2]))  # Remove last two words
        return variations

    # Function to remove abbreviations and extra space
    def remove_abbreviation_and_space(text):
        return re.sub(r'\([^()]*\)\s*', '', text).strip()

    # Remove abbreviations and extra space in the policy name
    policy_name_clean = remove_abbreviation_and_space(policy_name)

    # Split the policy name into words
    words = policy_name_clean.split()

    # Generate variations by removing one or two last words
    variations.extend(generate_variations(words))

    # Include original policy name without abbreviations
    variations.append(policy_name_clean)
    set_var=set(variations)
    variations=list(set_var)
    return variations

# Sample policy names
policy_names = [
    "National AIDS Control Organisation (NACO) Internship Programme",
    "Quality Improvement Programme (QIP)",
    "Livestock Health and Diseases Control",
    "Financial Assistance For Widows, Divorced, Separated / Destitute Deserted Or Abandoned Women",
    "Ramalingaswami Re-Entry Fellowship",
    "Financial Assistance For Development Of Buddhist/Tibetan Arts And Culture",
    "Institutional Support for Development and Marketing of Tribal Products/Produce",
    "Innovation In Science Pursuit For Inspired Research (INSPIRE) - Internship",
    "National Action Plan for Skill Development of Persons with Disabilities",
    "Education Loan Scheme",
    "Indira Gandhi National Widow Pension Scheme",
    "Maulana Abul Kalam Azad Trophy Scheme",
    "Biotechnology Ignition Grant Scheme",
    "Atal Mission For Rejuvenation And Urban Transformation",
    "Prime Minister's Scholarship Scheme for Railway Protection Force",
    "Coffee Development Programme in North Eastern Region: Quality Upgradation / Certification: Supply of Baby Pulpers",
    "Internship Scheme For PG/Research Students In Directorate General Of Trade Remedies (DGTR)",
    "AICTE - Research Promotion Scheme (RPS)",
    "Delhi Arogya Nidhi",
    "Swarnajayanti Fellowships Scheme",
    "MeitY - Visvesvaraya PhD Scheme For Electronics & IT",
    "National Means-Cum-Merit Scholarship Scheme",
    "Reimbursement of membership Fee in Government promoted E-Commerce Portals",
    "Pradhan Mantri Gram Sadak Yojana",
    "ICAR Post Matric Scholarship For Scheduled Caste / Scheduled Tribes Candidates",
    "Pre-Matric Scholarships to the Children of Those Engaged in Occupations Involving Cleaning and Prone to Health Hazards",
    "Central Sector Scheme of National Fellowship for Providing Fellowship to Scheduled Caste Students to Pursue M.Phil. & PhD",
    "Swachh Bharat Mission – Grameen PHASE I",
    "Indian Community Welfare Fund",
    "Saansad Adarsh Gram Yojana",
    "Tagore National Fellowship for Cultural Research",
    "Fellowship For Superannuated Faculty Members",
    "Indo-French Visiting/Mobility Programme for Women in Science",
    "VAIshwik BHArtiya Vaigyanik (VAIBHAV) Fellowship",
    "AICTE-IDEA (Idea Development, Evaluation & Application) LAB SCHEME",
    "Pragati Scholarship Scheme For Girl Students (Technical Diploma)",
    "AICTE-Margdarshan scheme"
]

# Generate variations for each policy name
all_variations = []
for policy_name in policy_names:
    variations = generate_policy_name_variations(policy_name)
    all_variations.extend(variations)

# Print all generated variations
print("Generated Variations:")
for idx, variation in enumerate(all_variations, start=1):
    print(f"{idx}. {variation}")


Generated Variations:
1. National AIDS Control Organisation Internship Programme
2. National AIDS Control Organisation
3. National AIDS Control Organisation (NACO) Internship Programme
4. National AIDS Control Organisation Internship
5. Quality Improvement Programme
6. Quality Improvement Programme (QIP)
7. Quality
8. Quality Improvement
9. Livestock Health and Diseases
10. Livestock Health and Diseases Control
11. Livestock Health and
12. Financial Assistance For Widows, Divorced, Separated / Destitute Deserted Or Abandoned Women
13. Financial Assistance For Widows, Divorced, Separated / Destitute Deserted Or
14. Financial Assistance For Widows, Divorced, Separated / Destitute Deserted Or Abandoned
15. Ramalingaswami Re-Entry
16. Ramalingaswami
17. Ramalingaswami Re-Entry Fellowship
18. Financial Assistance For Development Of Buddhist/Tibetan Arts And Culture
19. Financial Assistance For Development Of Buddhist/Tibetan Arts And
20. Financial Assistance For Development Of Buddhist/Tibe

In [14]:
#Retrieveing tweets from the Apify Client- Twitter Search
import pandas as pd
from apify_client import ApifyClient

schemes_df=pd.read_csv("min_20_unique_policy_names_according_to _state.csv")
policy_names=schemes_df['Policy name']


tweets={
    'Policy Name':[],
    'Policy Name Variant':[],
    'Id':[],
    'Date':[],
    'Url':[],
    'Full Text':[],
    'Hashtags':[],
    'Views Count':[],
    'Favourite Count':[],
    'Retweet Count':[],
    'Reply Count':[],
    'Language':[],
    'Image Urls':[],
    'Video Urls':[],
    'User Mentions':[],
    'User Profile Id':[],
    'User Description':[],
    'User Name':[],
    'User Followers Count':[]
}

client = ApifyClient('apify_api_0O0TFcKNUIzAWyO1VKgeU3BWxvBtw41Yeoez')

for policy_name in policy_names:
  policy_name_variations=generate_policy_name_variations(policy_name)
  for policy_name_variant in policy_name_variations:
    raw_input={
        "addUserInfo": True,
        "maxRequestRetries": 6,
        "maxTweets": 200,
        "scrapeTweetReplies": True,
        "searchMode": "live",
        "searchTerms": [
            policy_name_variant
        ]
    }
    run = client.actor("microworlds/twitter-scraper").call(run_input=raw_input)
    c=0
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
      if 'created_at' in item and 'url' in item and 'id_str' in item and 'favorite_count' in item and 'full_text' in item:
          c += 1
          tweets['Policy Name'].append(policy_name)
          tweets['Policy Name Variant'].append(policy_name_variant)
          
          if 'created_at' in item:
              tweets['Date'].append(item['created_at'])
          else:
              tweets['Date'].append(None)
              
          tweets['Url'].append(item['url'])
          tweets['Id'].append(item['id_str'])
          tweets['Favourite Count'].append(item['favorite_count'])
          tweets['Full Text'].append(item['full_text'])
          tweets['Language'].append(item['lang'])
          
          if 'reply_count' in item:
              tweets['Reply Count'].append(item['reply_count'])
          else:
              tweets['Reply Count'].append(None)
              
          if 'retweet_count' in item:
              tweets['Retweet Count'].append(item['retweet_count'])
          else:
              tweets['Retweet Count'].append(None)
              
          if 'views_count' in item:
              tweets['Views Count'].append(item['views_count'])
          else:
              tweets['Views Count'].append(None)
              
          hashtags = [i['text'] for i in item['entities']['hashtags']] if 'hashtags' in item['entities'] else []
          tweets['Hashtags'].append(hashtags)
          
          image_urls = [i['media_url_https'] for i in item['entities']['media'] if i['type'] == 'photo'] if 'media' in item['entities'] else []
          tweets['Image Urls'].append(image_urls)
          
          video_urls = [i['video_info']['variants'][1]['url'] for i in item['entities']['media'] if i['type'] == 'video'] if 'media' in item['entities'] else []
          tweets['Video Urls'].append(video_urls)
          
          user_mentions = [i['screen_name'] for i in item['entities']['user_mentions']] if 'user_mentions' in item['entities'] else []
          tweets['User Mentions'].append(user_mentions)
          
          tweets['User Profile Id'].append(item['user_id_str'])
          tweets['User Description'].append(item['user']['description'])
          tweets['User Name'].append(item['user']['screen_name'])
          tweets['User Followers Count'].append(item['user']['followers_count'])
      else:
          # Skip this item if any of the required fields are missing
          continue
    print("Total tweets retrieved for",policy_name_variant," = ",c)

# Now 'tweets' contains the data from the items that had all the required fields

tweets_df=pd.DataFrame(tweets)
tweets_df.to_csv("raw_tweets.csv", index=False)
print("Tweets CSV created")

tweets_df.head(5)


Total tweets retrieved for National AIDS Control Organisation Internship Programme  =  1
Total tweets retrieved for National AIDS Control Organisation  =  20
Total tweets retrieved for National AIDS Control Organisation (NACO) Internship Programme  =  1
Total tweets retrieved for National AIDS Control Organisation Internship  =  2
Total tweets retrieved for Quality Improvement Programme  =  20
Total tweets retrieved for Quality Improvement Programme (QIP)  =  20
Total tweets retrieved for Quality  =  20
Total tweets retrieved for Quality Improvement  =  20
Total tweets retrieved for Livestock Health and Diseases  =  19
Total tweets retrieved for Livestock Health and Diseases Control  =  20
Total tweets retrieved for Livestock Health and  =  20
Total tweets retrieved for Ramalingaswami Re-Entry  =  20
Total tweets retrieved for Ramalingaswami  =  20
Total tweets retrieved for Ramalingaswami Re-Entry Fellowship  =  20
Total tweets retrieved for Financial Assistance For Development Of Bud

,Policy Name,Policy Name Variant,Id,Date,Url,Full Text,Hashtags,Views Count,Favourite Count,Retweet Count,Reply Count,Language,Image Urls,Video Urls,User Mentions,User Profile Id,User Description,User Name,User Followers Count
0,National AIDS Control Organisation (NACO) Inte...,National AIDS Control Organisation Internship ...,1759914357943910795,Tue Feb 20 12:14:15 +0000 2024,https://twitter.com/tnsfrbc/status/17599143579...,Stipend INR.8000 | National AIDS Control Organ...,"[tnsfrbc, naco, internships, undergraduates, p...",50,1,1,0,en,[],[],[],1346692616704774146,The Core demand of SFRBC,tnsfrbc,169
1,National AIDS Control Organisation (NACO) Inte...,National AIDS Control Organisation,1781269484575338928,Fri Apr 19 10:31:54 +0000 2024,https://twitter.com/NACOINDIA/status/178126948...,People infected with HIV should take treatment...,"[KnowFacts, KnowHIV, IndiaFightsHIVandSTI, Cor...",91,4,1,0,en,[],[https://video.twimg.com/ext_tw_video/17812694...,[MoHFW_INDIA],1035019353190621185,This is official twitter account of National A...,NACOINDIA,7760
2,National AIDS Control Organisation (NACO) Inte...,National AIDS Control Organisation,1780904732846075959,Thu Apr 18 10:22:30 +0000 2024,https://twitter.com/NACOINDIA/status/178090473...,"मिथक नहीं, तथ्य फैलाएं।\n\nHIV का संक्रमण साथ ...","[KnowFacts, KnowHIV, IndiaFightsHIVandSTI, Cor...",162,6,2,0,hi,[https://pbs.twimg.com/media/GLcLZ1UaMAAAEPM.jpg],[],[MoHFW_INDIA],1035019353190621185,This is official twitter account of National A...,NACOINDIA,7760
3,National AIDS Control Organisation (NACO) Inte...,National AIDS Control Organisation,1780189621395292185,Tue Apr 16 11:00:54 +0000 2024,https://twitter.com/NACOINDIA/status/178018962...,Condoms offer three layers of protection:\n1. ...,"[KnowFacts, KnowHIV, IndiaFightsHIVandSTI, Con...",187,12,2,0,en,[https://pbs.twimg.com/media/GLSA574aAAAZ_6z.jpg],[],[MoHFW_INDIA],1035019353190621185,This is official twitter account of National A...,NACOINDIA,7760
4,National AIDS Control Organisation (NACO) Inte...,National AIDS Control Organisation,1779914215429685485,Mon Apr 15 16:46:32 +0000 2024,https://twitter.com/StigmabaseO/status/1779914...,HIV Fighting Stigma} Red Ribbon Express: Missi...,[],8,0,0,0,en,[],[],[],138418838,North & South America — Australia & NZ — HIV A...,StigmabaseO,4178


In [17]:
tweets_df.info()
cleaned_tweets_df=tweets_df.drop_duplicates(subset=['Policy Name', 'Full Text'])
cleaned_tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3181 entries, 0 to 3180
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Policy Name           3181 non-null   object
 1   Policy Name Variant   3181 non-null   object
 2   Id                    3181 non-null   object
 3   Date                  3181 non-null   object
 4   Url                   3181 non-null   object
 5   Full Text             3181 non-null   object
 6   Hashtags              3181 non-null   object
 7   Views Count           3181 non-null   int64 
 8   Favourite Count       3181 non-null   int64 
 9   Retweet Count         3181 non-null   int64 
 10  Reply Count           3181 non-null   int64 
 11  Language              3181 non-null   object
 12  Image Urls            3181 non-null   object
 13  Video Urls            3181 non-null   object
 14  User Mentions         3181 non-null   object
 15  User Profile Id       3181 non-null   

In [ ]:
#Retrieveing Quora Posts from the Quora Search

search_term="Ramalingaswami Re-Entry Fellowship"

raw_input={
    "content": "JSON",
    "limit": 1000,
    "query": search_term,
    "search_type": "all_types",
    "timing": "all_times"
}

# Run the Actor and wait for it to finish
run = client.actor("jupri/quora-scraper").call(run_input=raw_input)


{'spans': [{'text': 'Brain Drain', 'modifiers': {'bold': True, 'italic': True}}], 'indent': 0, 'type': 'plain'} : [{'spans': [{'text': 'Brain Drain', 'modifiers': {'bold': True, 'italic': True}}], 'indent': 0, 'type': 'plain'}, {'spans': [{'text': 'Brain Drain is a economic term, where'}, {'text': 'educated people of any country working in another country coming back to work in their home country', 'modifiers': {'italic': True}}, {'text': '.'}], 'indent': 0, 'type': 'ordered-list'}, {'spans': [{'text': 'Over'}, {'text': '1,000 Indian scientists', 'modifiers': {'bold': True}}, {'text': 'working'}, {'text': 'abroad', 'modifiers': {'bold': True}}, {'text': 'have'}, {'text': 'returned to India', 'modifiers': {'bold': True}}, {'text': 'in the last'}, {'text': 'two-three years,', 'modifiers': {'bold': True}}, {'text': 'said'}, {'text': 'Ministry for Science & Technology and Earth Sciences', 'modifiers': {'bold': True}}], 'indent': 0, 'type': 'ordered-list'}, {'spans': [{'text': 'According to

In [ ]:
#Retrieveing tweets from the Apify Client- Quora Search
import pandas as pd
from apify_client import ApifyClient

schemes_df=pd.read_csv("min_20_unique_policy_names_according_to _state.csv")
policy_names=schemes_df['Policy name']

quora_posts = {
    'Policy Name': [],
    'Policy Name Variant': [],
    'Id': [],
    'Date': [],
    'Url': [],
    'Title': [],
    'Full Text': [],
    'Type': [],
    'Upvotes Count': [],
    'Views Count': [],
    'Shares Count': [],
    'Comments Count': [],
    'Image Urls': [],
    'Video Urls': [],
    'User Id': [],
    'User Name': [],
}

for policy_name in policy_names:
    policy_name_variations = generate_policy_name_variations(policy_name)
    for policy_name_variant in policy_name_variations:
        raw_input = {
            "content": "JSON",
            "limit": 1000,
            "query": policy_name_variant,
            "search_type": "all_types",
            "timing": "all_times"
        }
        run = client.actor("jupri/quora-scraper").call(run_input=raw_input)
        c=0
        for item in client.dataset(run["defaultDatasetId"]).iterate_items():
          c += 1
          search = item.get('__answer') if item.get('type') == 'question' else item

          full_text = ""
          image_urls = []
          video_urls = []

          for j in search.get('content', []):
              for k in j.get('spans', []):
                  if 'text' not in k:
                      if 'modifiers' in k:
                          for l in k['modifiers']:
                              if l == 'image':
                                  image_urls.append(k['modifiers']['image'])
                              if l == 'video':
                                  video_urls.append(k['modifiers']['video'])
                      continue
                  full_text += k['text'] + ' '

          title = item.get('title') if item.get('type') == 'question' else (item.get('content', [{}])[0].get('spans', [{}])[0].get('text', None))

          quora_posts['Title'].append(title)
          quora_posts['Full Text'].append(full_text)
          quora_posts['Id'].append(search.get('id'))
          quora_posts['Type'].append(search.get('type'))
          quora_posts['Url'].append(search.get('url'))
          quora_posts['Image Urls'].append(image_urls)
          quora_posts['Video Urls'].append(video_urls)
          
          user_name = search['user']['names'][0].get('first') if 'names' in search['user'] else None
          if user_name and 'last' in search['user']['names'][0]:
              user_name += search['user']['names'][0]['last']
          quora_posts['User name'].append(user_name)
          
          quora_posts['User Id'].append(search['user'].get('id'))
          quora_posts['Comments Count'].append(search['stats'].get('comments'))
          quora_posts['Upvotes Count'].append(search['stats'].get('upvotes'))
          quora_posts['Views Count'].append(search['stats'].get('views'))
          quora_posts['Shares Count'].append(search['stats'].get('shares'))

          quora_posts['Date'].append(search.get('ts_created'))

        print("Total Quora posts retrieved for", policy_name_variant, " = ", c)


quora_posts_df=pd.DataFrame(quora_posts)
quora_posts_df.to_csv("raw_quora_posts.csv", index=False)
print("Quora Posts CSV created")
quora_posts_df.head(5)


Quora Posts CSV created
Total Quora Posts retrieved =  4


,Id,Date,Url,Title,Full Text,Type,Upvotes Count,Views Count,Shares Count,Comments Count,Image Urls,Video Urls,User Id,User name
0,80644619,2018-04-22T14:22:30.637110Z,https://www.quora.com/What-are-some-prominent-...,What are some prominent examples of Indian bra...,"Brain Drain Brain Drain is a economic term, wh...",answer,1,3091,0,0,[https://qph.cf2.quoracdn.net/main-qimg-aa3682...,[],168640941,MayurDhokchaule
1,1477743656698452,2023-04-04T05:20:01.164844Z,https://www.quora.com/What-are-some-fellowship...,What are some fellowships for research availab...,1. Khorana Program for Scholars 2. Fulbright F...,answer,0,49,0,0,[],[],233280385,The Google Bot
2,8309576,2020-09-23T09:42:45.997903Z,https://generalknowledgelive.quora.com/Schemes...,Schemes for Indian researchers residing in for...,Schemes for Indian researchers residing in for...,post,0,958,2,0,[],[],1292556795,ShivPrasad
3,137017811,2023-11-26T08:03:07.711494Z,https://azeezunv.quora.com/The-significance-of...,The significance of Fellowships in research an...,The significance of Fellowships in research an...,post,0,15,0,0,[https://qph.cf2.quoracdn.net/main-qimg-2168bf...,[],506976693,Azeez


In [ ]:
c=0
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    c+=1

    for i in item.keys():
        print(i,":",item[i])

type : question
id : 8206117
slug : What-are-some-prominent-examples-of-Indian-brain-drain-becoming-brain-gain
url : https://www.quora.com/What-are-some-prominent-examples-of-Indian-brain-drain-becoming-brain-gain
title : What are some prominent examples of Indian brain drain becoming brain gain?
ts_created : 2016-05-14T18:06:42.046621Z
ts_followed : 2019-05-27T00:59:24.234869Z
user : {'type': 'user', 'id': 66265543}
stats : {'answers': 3, 'followers': 2, 'comments': 0}
share : {'facebook': 'https://www.quora.com/What-are-some-prominent-examples-of-Indian-brain-drain-becoming-brain-gain?ch=1&oid=8206117&share=440bbb93&target_type=question', 'twitter': 'https://www.twitter.com/intent/tweet?lang=en&related=quora&text=Question+on+%40Quora%3A+What+are+some+prominent+examples+of+Indian+brain+drain+becoming+brain+gain%3F&url=https%3A%2F%2Fwww.quora.com%2FWhat-are-some-prominent-examples-of-Indian-brain-drain-becoming-brain-gain%3Fch%3D2%26oid%3D8206117%26target_type%3Dquestion', 'line': 'htt

In [ ]:
print(c)

4
